In [1]:
import numpy as np
import random
import time
from itertools import combinations
total_bandwidth = None
total_num_subsets =None
set_all_cameras = None
subsets = None
subsets_acc = None
subsets_bandwidth = None
random.seed(1000)

In [2]:
def compute_subset_bandwidth(subset):
    BANDWIDTH = 10
    n = len(subset)
    if (n==1):
        return  n*BANDWIDTH/2
    bandwidth = n*(n-1)*BANDWIDTH/2 + n*BANDWIDTH/2
    return bandwidth

In [3]:
def check_camera_coverage(solution,subsets,set_all_cameras):
    set_ = []
    
    for s in solution:
        sub = subsets[s]
        for c in sub:
            set_.append(c)
    set_ = set(set_)
    return set_==set(set_all_cameras)
#check_camera_coverage([[1,1,1],[2,2,2]],{0,1,2})

In [4]:
def compute_subset_accuracy(subset,set_all_accs):
    ## Will implement later
    n = len(subset)
    sum_ = 0
    for s in subset:
        sum_ += set_all_accs[s]
    sum_ = sum_/n
    return sum_

In [5]:
def process_data(filepath):
    
    file_ = open(filepath, "r")
    lines = file_.readlines() 
    set_all_accs =  [int(x.rstrip()) for x in lines]
    total_bandwidth = set_all_accs[0]
    set_all_accs = set_all_accs[1:]
    set_all_cameras = [i for i in range(len(set_all_accs))]
    subsets = []
    subsets_bandwidth = []
    subsets_acc = []
    for i in range(1,5):
        combs = combinations(range(len(set_all_cameras)), i)
        for subset in combs:
            subsets.append(subset)
            subsets_bandwidth.append(compute_subset_bandwidth(subset))
            subsets_acc.append(compute_subset_accuracy(subset,set_all_accs))
            

    total_num_subsets = [i for i in range(len(subsets))]
    #print(subsets)
    
    ''' We need to set out how the inputs will be.
    First, how would we set out our subsets in the dataset. 
    Will it be randomly sized sets (perhaps based on some other factors like geography)?
    Or will it just be pairs or triplets, each with a total bandwidth cost as a constraint?

    Second, should the accuracy cost which we are max be associated with each subset, or computed dynamically for each subset?
    But even if computed dynamically, we still need an accuracy associated with each camera element or subset to do the exponentially decaying number
    I've tried to do that below.
    '''

    ''' This should return a list or array of subsets; 
    and another 1 or 2 lists or arrays of associated costs/constraints; or
    use a dictionary where the keys are the subsets with bandwidth and accuracy keys as the associated values'''

    ''' Here I've assumed that this takes the data file, and returns the total number of subsets, a set of all cameras, the list of subsets, the accuracy of the subsets and the bandwidth'''
    return total_bandwidth,total_num_subsets, set_all_cameras, subsets, subsets_acc, subsets_bandwidth


'''
100

1,2,3,4,5...100

Subset Bandwidth
1,2,3  100
2,3,4  200


Camera Acc
1       0.5
2       0.7
....

'''

#total_bandwidth,total_num_subsets, set_all_cameras, subsets, subsets_acc, subsets_bandwidth = process_data("input.txt")

'\n100\n\n1,2,3,4,5...100\n\nSubset Bandwidth\n1,2,3  100\n2,3,4  200\n\n\nCamera Acc\n1       0.5\n2       0.7\n....\n\n'

In [6]:
def greedy_randomized_construction(alpha, total_num_subsets, set_all_cameras):

    # set of subsets in solution
    MAX = 1000000
    solution = set()
    #solution_camera_attributes = set() # this is actually a set of subsets or set of cameras
    solution_camera_attributes = []
    candidates = set(total_num_subsets) # this is actually a set of all the subset that we have by index

    # we need to determine the range of the RCL with the alpha and some metric
    # for example, we can use the ratio of accuracy/bandwidth which is what I have done here
    # or we can just directly use accuracy
    # this controls the range of candidates that can go into the RCL
    # While we have not covered all cameras, run the parts below
    while not check_camera_coverage(solution, subsets,set_all_cameras):
        # add to RCL while not all cameras not covered yet
        ratio = dict()

        # Calculate the ratio for every subset in the candidate set
        # Basically we run through every subset possible (i.e. each subset is a candidate)
        # See if this subset is already in the solution_camera_attributes
        # if it is, then we compute a ratio
        # This ratio is used to determine the GRASP alpha
        for i in candidates:
            # basically we want to check through whether the subset has the camera or not
            #attribute_to_add = len(subsets[i] - solution_camera_attributes)
            #print(attributes_to_add)

            #if attribute_to_add > 0:
            if i not in solution:
                # we need to see whether this greedy metric works
                ratio[i] = subsets_acc[i] / subsets_bandwidth[i]
                #print(ratio[i])
                # ratio[i] = subsets_acc[i] /total accuracy

        c_min = min(ratio.values())
        c_max = max(ratio.values())
        
        RCL = [i for i in candidates if i in ratio.keys() and ratio[i] <= c_min + alpha * (c_max - c_min)]
        #print(len(candidates),len(RCL))
        selected_index = random.choice(RCL)

        # take out that specific subset's index from the set of candidates (subset)
        candidates -= {selected_index}
        # add the index of the subset to the solution
        solution.add(selected_index)
        # add the subset to the solution_camera_attributes list
        solution_camera_attributes.append(subsets[selected_index]) 
    return solution
        
#solution = greedy_randomized_construction(0.1, total_num_subsets, set_all_cameras)



In [7]:
def compute_accuracy(solution):
    sum_ = 0
    for s in solution:
        sum_+=subsets_acc[s]
    return sum_
        

In [8]:
def compute_bandwidth(solution):
    sum_ = 0
    for s in solution:
        sum_+=subsets_bandwidth[s]
    return sum_

In [9]:
def is_feasible(solution,total_bandwidth):
    return check_camera_coverage(solution,subsets,set_all_cameras) and compute_bandwidth(solution) <= total_bandwidth

In [10]:
def local_search(solution):
    # we take the current solution and the unused subsets
    # iterate and swap if it leads to a better solution in terms of higher accuracy

    # Subsets not in solution
    subsets_not_in_solution = set([s for s in total_num_subsets if s not in solution])
    #print(solution)
    best_accuracy = compute_accuracy(solution)
    #print(best_accuracy)
    best_solution_set = solution.copy()

    temp_solution = solution.copy()
    for s_out in solution:
        for s_in in subsets_not_in_solution:
            # recall we want higher accuracy so we swap in s_in if it has higher acc
            if subsets_acc[s_in] > subsets_acc[s_out]:
                # do the swap if we can get better accuracy based on simple comparison of the 2 subsets accuracy
                temp_solution.update({s_in})
                temp_solution.difference_update({s_out})

                accuracy = compute_accuracy(temp_solution)

                if accuracy > best_accuracy and is_feasible(temp_solution,total_bandwidth):
                    # update the current best accuracy and solution
                    best_accuracy = accuracy
                    best_solution_set = temp_solution
                    #print(best_accuracy,compute_bandwidth(temp_solution))
    return best_solution_set
#local_search(solution)

In [11]:
def repair_unfeasible(solution):
    subsets_not_in_solution = set([s for s in total_num_subsets if s not in solution])
    best_bandwidth = compute_bandwidth(solution)

    #best_bandwidth = compute_bandwidth(solution)
    #best_solution_set = solution.copy()

    temp_solution = solution.copy()

    for s_out in solution:
        subs = subsets[s_out]
        temp_solution.difference_update({s_out})
        if len(subs)>1:
            for s_in in subs: 
                temp_solution.update({s_in})
            #if bandwidth < best_bandwidth:

            if is_feasible(temp_solution,total_bandwidth):
                return temp_solution
            #solution = temp_solution
    return None

In [12]:
def path_relinking(initial_solution, final_solution):

    # set of subsets that are only in either the initial or final solution
    subsets_not_intersection = initial_solution.symmetric_difference(final_solution)
    #print("DEBUG: ",initial_solution, final_solution)
    
    if len(subsets_not_intersection) == 0:
        return initial_solution
    acc_init_sol = compute_accuracy(initial_solution)
    acc_final_sol = compute_accuracy(final_solution)

    best_acc = max(acc_init_sol, acc_final_sol)

    if acc_init_sol > acc_final_sol:
        best_solution = initial_solution
    else:
        best_solution = final_solution

    current_solution = initial_solution.copy()

    best_dif_acc = 0
    best_change_acc = best_acc

# Basically we go though all subsest that are not at intersection and progessively add them to the initial solution
    while len(subsets_not_intersection) > 0:
        best_s = None
        for s in subsets_not_intersection:
            if s in current_solution:
                dif_acc = -subsets_acc[s]
            else:
                dif_acc = subsets_acc[s]

            current_solution.symmetric_difference_update({s})

            if (dif_acc > best_dif_acc) and is_feasible(current_solution,total_bandwidth):
                best_s = s
                best_dif_acc = dif_acc

            #current_solution.symmetric_difference_update({s})
        if best_s!=None:
            current_solution.symmetric_difference_update({best_s})

        best_change_acc += best_dif_acc

        if best_change_acc > best_acc:
            best_acc = best_change_acc
            best_solution = current_solution.copy()
        if best_s == None:
            return best_solution
        else:
            subsets_not_intersection.remove(best_s)

    return best_solution



In [13]:
def GRASP(num_iterations, alpha, num_elite, path_relinking_flag=True,debug=True):
    best_acc = 0
    best_sol = None
    # list of solutions that we have gone through
    S = []
    elite_S = []
    iteration = num_iterations
    while iteration > 0:
        iteration -= 1
        solution = greedy_randomized_construction(alpha, total_num_subsets,set_all_cameras)
        solution = local_search(solution)
        #print("F",solution)
        if not is_feasible(solution,total_bandwidth):
            solution = repair_unfeasible(solution)
            #print("D",solution)
        S.append(solution)
        #print("Best Accuracy: ",best_acc)
        acc = compute_accuracy(S[-1])
        if acc > best_acc:
            best_acc = acc
            best_sol = S[-1]
            if len(elite_S) < num_elite:
                elite_S.append(best_sol)
            else:
                elite_S[0:num_elite-1] = elite_S[1:num_elite]
                elite_S[num_elite-1] = best_sol
        if path_relinking_flag:

            # num_sample_pool
            #pool = random.sample(range(len(S)), num_sample_pool)
            if (len(elite_S)>0):
                random_elite_idx = random.randint(0,len(elite_S)-1)
                elite_solution = elite_S[random_elite_idx]

                # How many pairs we do here will depend on the num of samples we take from the solution list thus far
                # elite solutions in S[]

                solution = path_relinking(solution, elite_solution)
                S.append(solution)
                acc_s_p = compute_accuracy(solution)
                
                if acc_s_p > best_acc:

                    best_acc = acc_s_p
                    best_sol = solution
        if (debug):
            print("Best Accuracy: ",best_acc, " Bandwidth: ",compute_bandwidth(best_sol))
    return best_acc, best_sol

#best_acc,best_sol = GRASP(100, 0.1, 10, False)
#print(best_acc,compute_bandwidth(best_sol))

In [16]:
#def evaluate(path_relinking_flag,alpha):
out_ = open("results/grasp_0.1.txt","w")
for i in range(14,15):
    for j in range(1):
        filepath = "test/test_"+str(i)+"_"+str(j)+".txt"
        total_bandwidth,total_num_subsets, set_all_cameras, subsets, subsets_acc, subsets_bandwidth = process_data(filepath)
        print(i,j,total_bandwidth)
        best_acc,best_sol = GRASP(100, 0.1, 10,True,True)
        out_.write(str(best_acc)+"\n")
        
out_.close() 
print("Done")
#evaluate(True,0.1)
##1360.75

14 0 625
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1178.0  Bandwidth:  550.0
Best Accuracy:  1197.4166666666665  Bandwidth:  600.0
Best Accuracy:  1220.8333333333335  Bandwidth:  590.0
Best Accuracy:  1220.8333333333335  Bandwidth:  590.0
Best Accuracy:  1220.8333333333335  Bandwidth:  590.0
Best Accuracy:  1220.8333333333335  Bandwidth:  590.0
Best Accuracy:  1220.8333333333335  Bandwidth:  590.0
Best Accuracy:  1220.8333333333335  Bandwidth:  590.0
Best Accuracy:  1220.8333